
# Generate transcript from Youtube video 
- Summarize Youtube's script by chapter creater configured. 
  - Create `markdown_note.md` with script and summary.
- Use [yt-dlp](https://pypi.org/project/yt-dlp/), [pydub](https://pypi.org/project/pydub/), [OpenAI-Whisper](https://pypi.org/project/openai-whisper/), [langchain](https://github.com/hwchase17/langchain), and [OpenAI](https://github.com/openai/openai-python) package. 


## Input variables

In [1]:
# Youtube video ID
youtube_video_id="heqlXmqrxaQ"

# Language of subscription 
language = "ko"


# Officially no way to get chapter automatically, 
# so copy and paste the time stamp and chapter in description of Youtube video.
# CAUTION: Timestamp 00:00 must be defined! 
chapter_part_in_description = """
00:00 시작
09:34 댓글 읽어보기: 강진구 기자에 대하여
34:58 불합리한 것은 모두 억압하고 착취하기 위한 것이다: ‘그놈정신’을 갖자
39:41 억압과 착취의 메커니즘과 국가경영의 지배구조
1:05:13 억압과 착취의 메커니즘: 뉴스타파 봉지욱 기자의 고발: 화천대유는 누구의 것
1:16:46 우리의 현실과 민주당의 인식
1:18:09 불합리한 기득권에 안주하고 있는 사람들: 문재인, 이낙연, 윤석열
1:22:43 진리: 사실부합성과 인간부합성의 조건
1:41:42 철학, 실력, 용기가 없으면 반드시 착취당하게 된다.
1:42:03 구체적이고도 시급한 치유방안
1:44:46 억압과 착취의 구조 → 대화와 토론의 구조
1:46:07 장엄한 대전환을 위하여 ‘그놈정신’을 발휘하자
1:53:28 부탁의 말씀
1:59:29 정리
"""

hint_to_fix = \
"""
강진국 -> 강진구
박대원 -> 박대용
도독성 -> 도덕성
윤류의식 -> 윤리의식 
알궏리를 -> 알권리를
어갑하고 착취하도록 -> 억압하고 착취하도록
일어쿽 저러고 -> 이러쿵 저러쿵
홍의 -> 혼외
나라구 -> 나락
도 탐사 -> 더탐사
개몽 -> 계몽
성충죠 -> 성취죠
석진 위원장 -> 혁신 위원장
성취혜층 모형 -> 성취예측모형
강준규 -> 강진구
홍의 자식 -> 혼외자식
고질범 51이라는 -> 고작 51일이라는
총화대에서 -> 청와대에서
문소 -> 문서
사회 통염 -> 사회통념
포난라서 -> 퍼날라서
학교 좀 -> 합격점
조직설리 -> 조직설계
있어 빌리티만 -> 있어빌리티만
문제인 -> 문재인
학교 좀 -> 합격점
조직설리 -> 조직설계
있어 빌리티만 -> 있어빌리티만
문제인 -> 문재인
집에 구조 -> 지배구조
구수도 -> 고스톱
민주공항 -> 민주공화국
앵글로스엑스 -> 앵글로색슨
게흐만모형 -> 게르만 모형
다른 100년 -> 다른백년
이인회경 -> 이래경
정영왕 -> 정영학
노치록 -> 녹취록
지해보 -> 제보 
나무기 -> 남욱이
로빈 -> 로비
"""

In [2]:
# Officially no way to get chapter automatically, 
# so we need to parse the text in description and set up the dictionary 
# [ (time_in_sec, chapter_title) ]
import re 
pattern = r'(\d+(:\d+){1,2})\s(.+)'
matches = re.findall(pattern, chapter_part_in_description)

def time_to_seconds(time):
    parts = time.split(':')
    seconds = int(parts[-1])
    minutes = int(parts[-2]) if len(parts) > 1 else 0
    hours = int(parts[-3]) if len(parts) > 2 else 0
    return hours * 3600 + minutes * 60 + seconds

chapters = [(time_to_seconds(time), title.strip()) for time, _, title in matches]


# Build up note with chapter and script under each chapter 

In [3]:
import os
import yt_dlp

# Download youtube video and extract audio file. 
def download(video_id: str) -> str:
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'paths': {'home': 'audio/'},
        'outtmpl': {'default': '%(id)s.%(ext)s'},
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }

    if os.path.exists(f'audio/{video_id}.mp3'):
        return ""

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([video_url])
        if error_code != 0:
            raise Exception('Failed to download video')

    return f'audio/{video_id}.mp3'

file_path = download(youtube_video_id)



In [4]:
# split audio file
import os
import time
from pydub import AudioSegment

if os.path.exists(file_path) or file_path != '':
    audio_data = AudioSegment.from_mp3(file_path)
    
    for i in range( len(chapters) ):

        current_time_in_sec, current_title = chapters[i]
        next_time_in_sec, next_title = chapters[i + 1] if i + 1 < len(chapters) else (None, None)

        current_time_in_ms = current_time_in_sec * 1000
        next_time_in_ms = next_time_in_sec * 1000 if next_time_in_sec is not None else 0

        if next_time_in_sec:
            splitted_audio_data = audio_data[current_time_in_ms:next_time_in_ms]
        else:
            splitted_audio_data = audio_data[current_time_in_ms:]

        splitted_audio_data.export(f'audio/{i}.mp3' , format="mp3")

In [5]:
# Transcribe the text from audio files.
import os
from datetime import datetime
from dotenv import load_dotenv

import whisper
import llm

# Setup OpenAI API key 
load_dotenv()

# Prepare the file path for text
text_file_folder_path = os.path.join( os.getcwd(), 'text')
if not os.path.exists( text_file_folder_path ):
    os.makedirs(text_file_folder_path) 


# You can adjust the model used here. Model choice is typically a tradeoff between accuracy and speed.
# All available models are located at https://github.com/openai/whisper/#available-models-and-languages.
whisper_model = whisper.load_model("small")

script_by_chapter = []
def transcribe(file_path: str) -> str:
    # `fp16` defaults to `True`, which tells the model to attempt to run on GPU.
    # For local demonstration purposes, we'll run this on the CPU by setting it to `False`.
    transcription = whisper_model.transcribe(file_path, fp16=False)
    return transcription['text'] # type: ignore

for i in range( len(chapters) ):
    current_time_in_sec, current_title = chapters[i]
    print( f'{datetime.now()} : {current_title} is transcripting... \n' )
    audio_file_path = os.path.join( os.getcwd(), 'audio', f'{i}.mp3' )

    text_file = os.path.join(text_file_folder_path, f'{i}.txt')

    if not os.path.exists(text_file):
        transcript = transcribe(audio_file_path)
        cleaned_transcript = llm.clean_up_sentence_punctuation_and_fix_errors(transcript, 
                                                                            hint_to_fix, 
                                                                            chunk_size=100, 
                                                                            verbose=True)

        merged_text = \
    f"""
    {transcript}

    --------

    {cleaned_transcript}
    """

        chapter_data = { 
                    "title": current_title,
                    "script": merged_text,
                    "summary" : ""
                    }
        
        script_by_chapter.append(chapter_data)

        # Save transcript file

        with open( text_file, "w") as file:
            file.write(chapter_data["script"])
    else:
        chapter_data = { 
            "title": current_title,
            "script": "",
            "summary" : ""
            }

        script_by_chapter.append(chapter_data)  



/Users/jinhoyoo/.pyenv/versions/3.11.4/envs/venvAI/lib/python3.11/site-packages/whisper/timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


2023-08-27 01:01:50.776012 : 시작 is transcripting... 

2023-08-27 01:01:50.776370 : 댓글 읽어보기: 강진구 기자에 대하여 is transcripting... 

2023-08-27 01:01:50.776427 : 불합리한 것은 모두 억압하고 착취하기 위한 것이다: ‘그놈정신’을 갖자 is transcripting... 

2023-08-27 01:01:50.776463 : 억압과 착취의 메커니즘과 국가경영의 지배구조 is transcripting... 

2023-08-27 01:01:50.776490 : 억압과 착취의 메커니즘: 뉴스타파 봉지욱 기자의 고발: 화천대유는 누구의 것 is transcripting... 

chunk_size = 100


> Entering new LLMChain chain...
Prompt after formatting:

    아래 텍스트에의 문장부호를 다 정리해줘. 그리고 소리나는대로 잘못 적은 단어를 고쳐줘. 내용상 나뉘어야 하면 줄바꿈도 해줘. 고유명사 틀린것도 수정해줘.
    그리고 아래 예제처럼 자주 틀리는 것들이 있으니 참고해서 수정해줘. 
강진국 -> 강진구
박대원 -> 박대용
도독성 -> 도덕성
윤류의식 -> 윤리의식 
알궏리를 -> 알권리를
어갑하고 착취하도록 -> 억압하고 착취하도록
일어쿽 저러고 -> 이러쿵 저러쿵
홍의 -> 혼외
나라구 -> 나락
도 탐사 -> 더탐사
개몽 -> 계몽
성충죠 -> 성취죠
석진 위원장 -> 혁신 위원장
성취혜층 모형 -> 성취예측모형
강준규 -> 강진구
홍의 자식 -> 혼외자식
고질범 51이라는 -> 고작 51일이라는
총화대에서 -> 청와대에서
문소 -> 문서
사회 통염 -> 사회통념
포난라서 -> 퍼날라서
학교 좀 -> 합격점
조직설리 -> 조직설계
있어 빌리티만 -> 있어빌리티만
문제인 -> 문재인
학교 좀 -> 합격점
조직설리 -> 조직설계
있어 빌리티만 -> 있어빌리티만
문제인 -> 문재인
집에 구조

In [6]:

# Temporary save data into file 
import os 
import json 

with open( "temp_script_by_chapter.json", "w") as file:
    file.write( json.dumps(script_by_chapter, indent=2, ensure_ascii=False) )
